Data Augmentation (logica e sintetica)

Obiettivo: aumentare i dati per le malattie con pochi esempi.
Esempi:
*	se una malattia ha <10 pazienti → duplichi o combini righe con sintomi simili,
*	regole tipo: se fever e nausea → aggiungi vomit,
*	randomizzazione sintomi tra pazienti della stessa malattia.

In [ ]:
import pandas as pd
import numpy as np
import random

# Caricamento dataset curato
df = pd.read_csv("/content/drive/MyDrive/BigData/dataset_curated_spark.csv")

# Identificazione delle colonne dei sintomi
symptom_cols = [col for col in df.columns if "Symptom_" in col]
# Conteggio delle occorrenze per malattia
disease_counts = df['Disease'].value_counts()
rare_diseases = disease_counts[disease_counts < 10].index.tolist()
rare_diseases


In [ ]:
def generate_synthetic_rows(df_sub, n_new=5):
    new_rows = []
    for _ in range(n_new):
        r1, r2 = df_sub.sample(2)
        new_symptoms = []
        for col in symptom_cols:
            s = r1[col] if random.random() > 0.5 else r2[col]
            new_symptoms.append(s)
        new_rows.append([df_sub['Disease'].iloc[0]] + new_symptoms)
    return new_rows


In [ ]:
synthetic_data = []

for disease in rare_diseases:
    subset = df[df['Disease'] == disease]
    if len(subset) >= 2:
        new_samples = generate_synthetic_rows(subset, n_new=(10 - len(subset)))
        synthetic_data.extend(new_samples)

# Creazione DataFrame sintetico
df_synth = pd.DataFrame(synthetic_data, columns=['Disease'] + symptom_cols)


In [ ]:
# Unione con dataset originale
df_augmented = pd.concat([df, df_synth], ignore_index=True)

# Ricrea symptom_count
df_augmented["symptom_count"] = df_augmented[symptom_cols].notna().sum(axis=1)

# Salva il dataset finale
df_augmented.to_csv("/content/drive/MyDrive/BigData/dataset_augmented.csv", index=False)
df_augmented.shape
